In [351]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

os.chdir("/home/william/Desktop")

In [352]:
class Analisis_Predictivo:

    def __init__(self,datos, predecir:str, predictoras = [],
                 modelo = None,train_size = 80,random_state = None):
        '''
        datos: Datos completos y listos para construir un modelo

        modelo: Instancia de una Clase de un método de clasificación(KNN,Árboles,SVM,etc).
        Si no especifica un modelo no podrá utilizar el método fit_n_review()

        predecir: Nombre de la variable a predecir

        predictoras: Lista de los nombres de las variables predictoras.
        Si vacío entonces utiliza todas las variables presentes excepto la variable a predecir.

        train_size: Proporción de la tabla de entrenamiento respecto a la original.

        random_state: Semilla aleatoria para la división de datos(training-testing).
        '''
        self.datos = datos
        self.predecir = predecir
        self.predictoras = predictoras
        self.modelo = modelo
        self.random_state = random_state
        if modelo != None:
            self.train_size = train_size
            self._training_testing()


    def _training_testing(self):
        if len(self.predictoras) == 0:
            X = self.datos.drop(columns=[self.predecir])
        else:
            X = self.datos[self.predictoras]

        y = self.datos[self.predecir].values

        train_test = train_test_split(X, y, train_size=self.train_size,
                                      random_state=self.random_state)
        self.X_train, self.X_test,self.y_train, self.y_test = train_test


    def fit_predict(self):
        if self.modelo != None:
            self.modelo.fit(self.X_train,self.y_train)
            return self.modelo.predict(self.X_test)

    def fit_predict_resultados(self, imprimir = True):
        if(self.modelo != None):
            y = self.datos[self.predecir].values
            prediccion = self.fit_predict()
            MC = confusion_matrix(self.y_test, prediccion)
            indices = self.indices_general(MC,list(np.unique(y)))
            if imprimir == True:
                for k in indices:
                    print("\n%s:\n%s"%(k,str(indices[k])))

            return indices

    def indices_general(self,MC, nombres = None):
        "Método para calcular los índices de calidad de la predicción"
        precision_global = np.sum(MC.diagonal()) / np.sum(MC)
        error_global = 1 - precision_global
        precision_categoria  = pd.DataFrame(MC.diagonal()/np.sum(MC,axis = 1)).T
        if nombres!=None:
            precision_categoria.columns = nombres
        return {"Matriz de Confusión":MC,
                "Precisión Global":precision_global,
                "Error Global":error_global,
                "Precisión por categoría":precision_categoria}

    def distribucion_variable_predecir(self):
        "Método para graficar la distribución de la variable a predecir"
        variable_predict = self.predecir
        data = self.datos
        colors = list(dict(**mcolors.CSS4_COLORS))
        df = pd.crosstab(index=data[variable_predict],columns="valor") / data[variable_predict].count()
        fig = plt.figure(figsize=(10,9))
        g = fig.add_subplot(111)
        countv = 0
        titulo = "Distribución de la variable %s" % variable_predict
        for i in range(df.shape[0]):
            g.barh(1,df.iloc[i],left = countv, align='center',color=colors[11+i],label= df.iloc[i].name)
            countv = countv + df.iloc[i]
        vals = g.get_xticks()
        g.set_xlim(0,1)
        g.set_yticklabels("")
        g.set_title(titulo)
        g.set_ylabel(variable_predict)
        g.set_xticklabels(['{:.0%}'.format(x) for x in vals])
        countv = 0
        for v in df.iloc[:,0]:
            g.text(np.mean([countv,countv+v]) - 0.03, 1 , '{:.1%}'.format(v), color='black', fontweight='bold')
            countv = countv + v
        g.legend(loc='upper center', bbox_to_anchor=(1.08, 1), shadow=True, ncol=1)

    def poder_predictivo_categorica(self, var:str):
        "Método para ver la distribución de una variable categórica respecto a la predecir"
        data = self.datos
        variable_predict = self.predecir
        df = pd.crosstab(index= data[var],columns=data[variable_predict])
        df = df.div(df.sum(axis=1),axis=0)
        titulo = "Distribución de la variable %s según la variable %s" % (var,variable_predict)
        g = df.plot(kind='barh',stacked=True,legend = True, figsize = (10,9), \
                    xlim = (0,1),title = titulo, width = 0.8)
        vals = g.get_xticks()
        g.set_xticklabels(['{:.0%}'.format(x) for x in vals])
        g.legend(loc='upper center', bbox_to_anchor=(1.08, 1), shadow=True, ncol=1)
        for bars in g.containers:
            plt.setp(bars, width=.9)
        for i in range(df.shape[0]):
            countv = 0
            for v in df.iloc[i]:
                g.text(np.mean([countv,countv+v]) - 0.03, i , '{:.1%}'.format(v), color='black', fontweight='bold')
                countv = countv + v


    def poder_predictivo_numerica(self,var:str):
        "Función para ver la distribución de una variable numérica respecto a la predecir"
        sns.FacetGrid(self.datos, hue=self.predecir, height=6).map(sns.kdeplot, var, shade=True).add_legend()


In [353]:
def transformToDataFrame(modelo):
    resultados = modelo.fit_predict_resultados(imprimir=True)
    precisionGlobal = pd.DataFrame({"Precisión Global" : [resultados['Precisión Global']]})
    errorGlobal = pd.DataFrame({"Error Global" : [resultados['Error Global']]})
    precisonCategoria = resultados['Precisión por categoría']
    finalDataFrame = precisionGlobal.join(errorGlobal).join(precisonCategoria)
    return finalDataFrame

In [354]:
potabilidad = pd.read_csv("potabilidad_V2.csv")
potabilidad = potabilidad.iloc[:,1:]
potabilidad["Potability"] = potabilidad["Potability"].astype('category')

El primer modelo sin estandarizar los datos  muestra unos resultados bastante malos, por que como podemos ver en el dataframe de abajo, aunque tiene una precision global interesante, la verdadera metrica, se ve en el "NO" y el "SI", y podemos ver que el valor de si es muy bajo

In [355]:
modelo = MLPClassifier(hidden_layer_sizes= (6,) , activation= "identity", solver="adam", random_state=0)
analisisModelo1 = Analisis_Predictivo(potabilidad,predecir= "Potability",modelo=modelo,
                                       train_size= 0.75, random_state=0)
transformToDataFrame(analisisModelo1)


Matriz de Confusión:
[[257  52]
 [161  33]]

Precisión Global:
0.5765407554671969

Error Global:
0.42345924453280315

Precisión por categoría:
         No        Si
0  0.831715  0.170103


,Precisión Global,Error Global,No,Si
0,0.576541,0.423459,0.831715,0.170103


Al igual que el primer modelo sin estandarizar los datos muestra unos resultados pesimos, el segundo modelo muestra unos resultados aun peores que el anterior, aunque parece que predice bien el "no", esto no es cierto por que al ver el valor del "SI",  podemos que lo que sucede es que categoriza casi todo como "NO", entonces por eso tiene ese valor tan alto

In [356]:
modelo2 = MLPClassifier(hidden_layer_sizes= (10,25) , activation="tanh", solver="sgd", random_state=0)
analisisModelo2 = Analisis_Predictivo(potabilidad,predecir= "Potability",modelo=modelo2,
                                       train_size= 0.75, random_state=0)
transformToDataFrame(analisisModelo2)


Matriz de Confusión:
[[309   0]
 [192   2]]

Precisión Global:
0.6182902584493042

Error Global:
0.3817097415506958

Precisión por categoría:
    No        Si
0  1.0  0.010309


,Precisión Global,Error Global,No,Si
0,0.61829,0.38171,1.0,0.010309


In [357]:
potabilidad.iloc[:,:-1] = StandardScaler().fit_transform(potabilidad.iloc[:,:-1])

Estas metricas pertenecen al modelo 1, con la estandarizacion de datos y aunque los resultados son malo al igual que en el anterior de acuerdo a la matriz de confusion podemos darnos cuenta, que realmente si mejoran bastante, con respecto al anterior

In [358]:
modelo = MLPClassifier(hidden_layer_sizes= (6,) , activation= "identity", solver="adam", random_state=0)
analisisModelo1 = Analisis_Predictivo(potabilidad,predecir= "Potability",modelo=modelo,
                                       train_size= 0.75, random_state=0)

transformToDataFrame(analisisModelo1)



Matriz de Confusión:
[[304   5]
 [184  10]]

Precisión Global:
0.6242544731610338

Error Global:
0.3757455268389662

Precisión por categoría:
         No        Si
0  0.983819  0.051546


,Precisión Global,Error Global,No,Si
0,0.624254,0.375746,0.983819,0.051546


En el caso del modelo 2 podemos ver que se mejora en el tema del si, aunque claramente se puede notar que es muy poco, si nota que la estadarizacion de datos es demasiado importante cuando estamos usando las redes neuronales

In [359]:
modelo2 = MLPClassifier(hidden_layer_sizes= (10,25) , activation="tanh", solver="sgd", random_state=0)
analisisModelo2 = Analisis_Predictivo(potabilidad,predecir= "Potability",modelo=modelo2,
                                       train_size= 0.75, random_state=0)
transformToDataFrame(analisisModelo2)


Matriz de Confusión:
[[306   3]
 [192   2]]

Precisión Global:
0.6123260437375746

Error Global:
0.3876739562624254

Precisión por categoría:
         No        Si
0  0.990291  0.010309


,Precisión Global,Error Global,No,Si
0,0.612326,0.387674,0.990291,0.010309
